# Создание системы анализа запросов

Раздел содержит готовый пример, который показывает, как использовать анализ запросов.
Пример демонстрирует:

- создание простого поискового движка;
- режим ошибки, который возникает при передаче необработанного вопроса пользователя в поиск;
- как анализ запросов может помочь исправить возникшую ошибку. 

:::note

Существует множество разных методик анализа запросов, которые не показаны в примере.

:::

Анализ запросов продемонстрирован на примере поиска по видео на YouTube-канале LangChain.

## Подготовка к работе

Установите зависимости и переменные окружения.

In [1]:
# %pip install -qU langchain langchain-community langchain-openai youtube-transcript-api pytube gigachain-chroma

При работе с этим руководством используется модель OpenAI.

In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

# Optional, uncomment to trace runs with LangSmith. Sign up here: https://smith.langchain.com.
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

### Загрузка документов

Используйте `YouTubeLoader` для загрузки транскрипций нескольких видео LangChain:

In [12]:
from langchain_community.document_loaders import YoutubeLoader

urls = [
    "https://www.youtube.com/watch?v=HAn9vnJy6S4",
    "https://www.youtube.com/watch?v=dA1cHGACXCo",
    "https://www.youtube.com/watch?v=ZcEMLz27sL4",
    "https://www.youtube.com/watch?v=hvAPnpSfSGo",
    "https://www.youtube.com/watch?v=EhlPDL4QrWY",
    "https://www.youtube.com/watch?v=mmBo8nlu2j0",
    "https://www.youtube.com/watch?v=rQdibOsL1ps",
    "https://www.youtube.com/watch?v=28lC4fqukoc",
    "https://www.youtube.com/watch?v=es-9MgxB-uc",
    "https://www.youtube.com/watch?v=wLRHwKuKvOE",
    "https://www.youtube.com/watch?v=ObIltMaRJvY",
    "https://www.youtube.com/watch?v=DjuXACWYkkU",
    "https://www.youtube.com/watch?v=o7C9ld6Ln-M",
]
docs = []
for url in urls:
    docs.extend(YoutubeLoader.from_youtube_url(url, add_video_info=True).load())

In [13]:
import datetime

# Добавление дополнительной метаданных: год публикации видео
for doc in docs:
    doc.metadata["publish_year"] = int(
        datetime.datetime.strptime(
            doc.metadata["publish_date"], "%Y-%m-%d %H:%M:%S"
        ).strftime("%Y")
    )

Заголовки загруженных видео хранятся в списке `docs`:

In [59]:
[doc.metadata["title"] for doc in docs]

['OpenGPTs',
 'Building a web RAG chatbot: using LangChain, Exa (prev. Metaphor), LangSmith, and Hosted Langserve',
 'Streaming Events: Introducing a new `stream_events` method',
 'LangGraph: Multi-Agent Workflows',
 'Build and Deploy a RAG app with Pinecone Serverless',
 'Auto-Prompt Builder (with Hosted LangServe)',
 'Build a Full Stack RAG App With TypeScript',
 'Getting Started with Multi-Modal LLMs',
 'SQL Research Assistant',
 'Skeleton-of-Thought: Building a New Template from Scratch',
 'Benchmarking RAG over LangChain Docs',
 'Building a Research Assistant from Scratch',
 'LangServe and LangChain Templates Webinar']

Вы также можете получить доступ к метаданным, связанным с каждым видео.
Каждый документ также имеет заголовок, количество просмотров, дату публикации и продолжительность:

In [60]:
docs[0].metadata

{'source': 'HAn9vnJy6S4',
 'title': 'OpenGPTs',
 'description': 'Unknown',
 'view_count': 7210,
 'thumbnail_url': 'https://i.ytimg.com/vi/HAn9vnJy6S4/hq720.jpg',
 'publish_date': '2024-01-31 00:00:00',
 'length': 1530,
 'author': 'LangChain',
 'publish_year': 2024}

Пример содержимого одного из документов:

In [61]:
docs[0].page_content[:500]

"hello today I want to talk about open gpts open gpts is a project that we built here at linkchain uh that replicates the GPT store in a few ways so it creates uh end user-facing friendly interface to create different Bots and these Bots can have access to different tools and they can uh be given files to retrieve things over and basically it's a way to create a variety of bots and expose the configuration of these Bots to end users it's all open source um it can be used with open AI it can be us"

### Индексирование документов

Каждый раз при извлечении данных нужно создавать индекс документов, к которым можно выполнять запросы.
Для индексирования документов используйте векторное хранилище.
Перед помещением документов в хранилище их на части, чтобы ваши запросы были более точными и краткими.

In [14]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
chunked_docs = text_splitter.split_documents(docs)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = Chroma.from_documents(
    chunked_docs,
    embeddings,
)

## Извлечение без анализа запросов

Вы можете выполнять поиск по сходству с запросом пользователя, для обнаружения подходящих фрагментов.

In [64]:
search_results = vectorstore.similarity_search("how do I build a RAG agent")
print(search_results[0].metadata["title"])
print(search_results[0].page_content[:500])

Build and Deploy a RAG app with Pinecone Serverless
hi this is Lance from the Lang chain team and today we're going to be building and deploying a rag app using pine con serval list from scratch so we're going to kind of walk through all the code required to do this and I'll use these slides as kind of a guide to kind of lay the the ground work um so first what is rag so under capoy has this pretty nice visualization that shows LMS as a kernel of a new kind of operating system and of course one of the core components of our operating system is th


Вы можете видеть удовлетворительный результат, соответствующий запросу пользователя.
Но что, если вам нужно искать результаты за определенный период времени?

In [65]:
search_results = vectorstore.similarity_search("videos on RAG published in 2023")
print(search_results[0].metadata["title"])
print(search_results[0].metadata["publish_date"])
print(search_results[0].page_content[:500])

OpenGPTs
2024-01-31
hardcoded that it will always do a retrieval step here the assistant decides whether to do a retrieval step or not sometimes this is good sometimes this is bad sometimes it you don't need to do a retrieval step when I said hi it didn't need to call it tool um but other times you know the the llm might mess up and not realize that it needs to do a retrieval step and so the rag bot will always do a retrieval step so it's more focused there because this is also a simpler architecture so it's always


Первый результат относится к 2024 году (хотя в запросе задан 2023 год) и не очень релевантен запросу.
Поскольку в данном случае выполняется простой поиск по содержимому документов, у вас нет возможности фильтровать результаты по каким-либо атрибутам документа.

Это лишь одна проблем, которые могут возникнуть. 
Рассмотрим, как анализ запросов может ее исправить.

## Анализ запросов

Используйте анализ запросов для улучшения результатов поиска.
Для этого нужно:

- определить схему запроса, которая будет содержать фильтр по датам;
- использовать модель, которая способна вызывать функци, для преобразования вопроса пользователя в структурированный запрос.

### Схема запроса

В представленном примере для даты публикации используются явные атрибуты, которые позволяют задать диапазон для фильтрации.

In [2]:
from typing import Optional

from langchain_core.pydantic_v1 import BaseModel, Field


class Search(BaseModel):
    """Search over a database of tutorial videos about a software library."""

    query: str = Field(
        ...,
        description="Similarity search query applied to video transcripts.",
    )
    publish_year: Optional[int] = Field(None, description="Year video was published")

### Генерация запросов

Для преобразования пользовательских вопросов в структурированные запросы в примере используется API вызова инструментов OpenAI.
Например, используется новый конструктор [ChatModel.with_structured_output()](/docs/how_to/structured_output) для передачи схемы модели и анализа вывода.

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

system = """You are an expert at converting user questions into database queries. \
You have access to a database of tutorial videos about a software library for building LLM-powered applications. \
Given a question, return a list of database queries optimized to retrieve the most relevant results.

If there are acronyms or words you are not familiar with, do not try to rephrase them."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
structured_llm = llm.with_structured_output(Search)
query_analyzer = {"question": RunnablePassthrough()} | prompt | structured_llm

/Users/bagatur/langchain/libs/core/langchain_core/_api/beta_decorator.py:86: LangChainBetaWarning: The function `with_structured_output` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


Посмотрите, какие запросы анализатор генерирует для ранее заданных поисковых вопросов:

In [4]:
query_analyzer.invoke("how do I build a RAG agent")

Search(query='build RAG agent', publish_year=None)

In [5]:
query_analyzer.invoke("videos on RAG published in 2023")

Search(query='RAG', publish_year=2023)

## Извлечение с анализом запросов

Используйте сгенерированные запросы для извлечения данных.

:::note

В примере указан инструмент `tool_choice="Search"`.
Это вынудит LLM вызвать только один инструмент, что приведет к тому, что у вас всегда будет один оптимизированный запрос для поиска.

Это не всегда так. В других руководствах вы найдете информацию о том, как поступать в ситуациях, когда не возвращается ни одного или возвращается несколько оптимизированных запросов.

:::

In [8]:
from typing import List

from langchain_core.documents import Document

In [16]:
def retrieval(search: Search) -> List[Document]:
    if search.publish_year is not None:
        # This is syntax specific to Chroma,
        # the vector database we are using.
        _filter = {"publish_year": {"$eq": search.publish_year}}
    else:
        _filter = None
    return vectorstore.similarity_search(search.query, filter=_filter)

In [17]:
retrieval_chain = query_analyzer | retrieval

Запустите полученную цепочку с входными данными, которые привели к проблеме, из примера выше и убедитесь, что она возвращает только результаты за указанный год:

In [20]:
results = retrieval_chain.invoke("RAG tutorial published in 2023")

In [21]:
[(doc.metadata["title"], doc.metadata["publish_date"]) for doc in results]

[('Getting Started with Multi-Modal LLMs', '2023-12-20 00:00:00'),
 ('LangServe and LangChain Templates Webinar', '2023-11-02 00:00:00'),
 ('Getting Started with Multi-Modal LLMs', '2023-12-20 00:00:00'),
 ('Building a Research Assistant from Scratch', '2023-11-16 00:00:00')]